# **Database Administration**

This notebook will guide you through some of the basic tasks conducted by a database administrator. You will be walked through the process of creating a database, adding users, managing roles, and granting permission. In addition to these tasks, we will also walk through backup and restoration procedures. Finally you will be guided through query optimization and data monitoring tasks. The goal of this notebook is to give a broad understanding of what a DBA (Database Administrator) is capable of. Please read the below carefully.

We will not be using our regular _production_ database for this assignment. You will instead use a database server that is running via docker. Before proceeding, follow the steps for setting up docker [here](https://otccis.gitbook.io/student-help-docs/docker). This change is to ensure that nobody accidentally causes issues with production. You will have to add the new database connection just like you did with the original one at the beginning of the term. The connection info is below:

**Server Name:**  localhost

**Authentication****:** SQL Login

**Login:** sa

**Password:** Your student id (XX01234567)

**Database:** master (default)

Once the information has been entered, click "Connect". If you have a connection issue, reach out to your instructor ASAP.

## **Database Creation and User Management**

Now that you are connected, we are going to start by creating a database. If you look at the list of databases in the connection pane you will see that you do not have a student database in this server. To create a database we will use the CREATE keyword with the DATABASE keyword. Edit the below script as directed and run it.

In [1]:
-- Create a new database named your student ID (xx01234567)
CREATE DATABASE ms0885011;


Commands completed successfully.

Total execution time: 00:00:00.206

The above cell creates an empty database. In our exercises so far we have operated out of one database and used prefixes on our tables names to differentiate them. In a real scenario we would create an individual database for each system and keep the tables separated.

Next we are going to create a login for an individual, and associate a user with that login. The terminolgy can be confusing, so here is a breakdown:

- Login- A username and password used to login to the SQL server where all the databases are kept
- User- A named object that is associated with a login. A user object outlines the security settings for a given login for each database. So, a login that has access to two databases in a server will have to users.

The below cell will create a login for the server:

In [2]:
--Change both of the below scripts to use your student id and a password of your choice (that you don't mind me seeing)
--Create a new login to the SQL server
CREATE LOGIN Ms0885011 WITH PASSWORD = 'RealPassword1234';



Commands completed successfully.

Total execution time: 00:00:00.005

This cell defines a user for the database that you just created. We can specify which database we want to effect by implementing the USE keyword. In the below cell we specify that we want to interact with the test database, so that we don't accidentally create a new user for Master. Remember, users are database specific.

In [3]:
--Add a user to the database you just created for the login established in the above line. Don't forget to update the database name.
USE Ms0885011
CREATE USER Ms0885011 FOR LOGIN Ms0885011

Commands completed successfully.

Total execution time: 00:00:00.006

To test that your login was succesful, you can open a new ADS window (File -\> New Window) <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">&nbsp;and add a connection, or change your current connection to validate that you are able to login. We will be continuing to use the admin login provided at the beginning of the notebook for the remainder of the exercise.</span>

Now that we have a user to manage, we will practice managing roles for a user. When a user is created they can't really interact with the database in a meaningful way because they have no permissions. We can assign pre-written sets of permissions by adding a user to a _role._ Roles describe the actions that members can or cannot take. We will give the new user the ability to write to the database, and read data from it, by adding them to the appropriate roles. This can be done by using an ALTER ROLE / ADD MEMBER statement.

In [4]:
--Don't forget to update the below script to use your student ID before running.

-- Add the user to the db_datareader role
ALTER ROLE db_datareader ADD MEMBER Ms0885011;

-- Add the user to db_datawriter
ALTER ROLE db_datawriter ADD MEMBER Ms0885011;

Commands completed successfully.

Total execution time: 00:00:00.006

Managing roles is much easier than managing individual users, but the needs of a user don't always fit into a specific role. In these cases we can use the GRANT keyword to give very specific permissions. To demonstrate we will need to add a table to our database:

In [4]:
--Create a table named after your student Id XX1234567_table
--No need to edit the attributes, just the table name
CREATE TABLE Ms0885011_table
(
    column_1 INT,
    column_2 INT
)

Commands completed successfully.

Total execution time: 00:00:00.005

Now that we have a table, we can grant our user access to different functions of the table so that they can interact with it:

In [6]:
-- Grant SELECT permission on a specific table
GRANT SELECT ON Ms0885011_table TO Ms0885011;

-- Grant INSERT, UPDATE, DELETE permissions on a specific table
GRANT INSERT, UPDATE, DELETE ON Ms0885011_table  TO Ms0885011;

Commands completed successfully.

Total execution time: 00:00:00.007

## **Backup and Recovery**

Backup and recovery are a key aspect of the database administrator role. There is always a chance of data being lost due to physical or digital circumstances, so it is crucial that data be backed up and stored in a way that it can be retrieved and restored as needed. Before we walk through the process of backing up data, let's add some data to the table we made earlier.

In [7]:
--Add data to table
INSERT INTO Ms0885011_table VALUES
(1,1),
(2,2),
(3,3);

(3 rows affected)

Total execution time: 00:00:00.009

The script for backing up a database is very straightforward, we just need to use the BACKUP keyword and specify a location on the disk (your computer). Database backups use the .bak file extension, so we will need to keep that in mind.

In [1]:
-- Full database backup
BACKUP DATABASE Ms0885011
TO DISK = 'C:\Backups'; --Will look something like C:\Backup\studentID_Database.bak (for windows users)

: Msg 3201, Level 16, State 1, Line 2
Cannot open backup device 'C:\Backups'. Operating system error 5(Access is denied.).

: Msg 3013, Level 16, State 1, Line 2
BACKUP DATABASE is terminating abnormally.

Total execution time: 00:00:00.024

To restore a database from a file on our local disk, we would use a script like this:

In [ ]:
-- Restore the database from backup
RESTORE DATABASE Ms0885011
FROM DISK = 'C:\Backup\Ms0885011_table.bak' --Unable to fully set up backup
WITH REPLACE;

## **Basic Performance Monitoring**

To close up our unit on database administration, we are going to go over some performance monitoring tools. I do not expect you to be experts on monitoring, I simply want you to be aware that these tools exists and know how to access them if needed.

Before continuing, check your connection to verify that you are connected to the database named after your student id and not master.

We will start with a simple script that shows all currently running queries for a database. Since you are likely the only person accessing the database, you will probably only have one record return.

In [2]:
-- View current running queries (no changes needed)
SELECT * FROM sys.dm_exec_requests;
GO

(63 rows affected)

Total execution time: 00:00:00.070

session_id,request_id,start_time,status,command,sql_handle,statement_start_offset,statement_end_offset,plan_handle,database_id,user_id,connection_id,blocking_session_id,wait_type,wait_time,last_wait_type,wait_resource,open_transaction_count,open_resultset_count,transaction_id,context_info,percent_complete,estimated_completion_time,cpu_time,total_elapsed_time,scheduler_id,task_address,reads,writes,logical_reads,text_size,language,date_format,date_first,quoted_identifier,arithabort,ansi_null_dflt_on,ansi_defaults,ansi_warnings,ansi_padding,ansi_nulls,concat_null_yields_null,transaction_isolation_level,lock_timeout,deadlock_priority,row_count,prev_error,nest_level,granted_query_memory,executing_managed_code,group_id,query_hash,query_plan_hash,statement_sql_handle,statement_context_id,dop,parallel_worker_count,external_script_request_id,is_resumable,page_resource,page_server_reads,dist_statement_id,label
1,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,46114,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47377,7,0x00000010004328C8,14,14,34,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
2,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,47375,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47377,3,0x0000001000412CA8,0,0,0,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
3,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,46114,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,10,0x000000100044A8C8,13,13,104,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
4,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,46114,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,8,0x000000100043A8C8,14,14,134,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
5,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,46114,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,0,0x0000001000078CA8,2,2,4,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
6,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,47375,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,5,0x0000001000422CA8,0,0,0,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
7,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,47375,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,2,0x000000100040ACA8,0,0,0,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
8,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,47375,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,1,0x0000001000403088,0,0,0,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
9,0,2024-11-22 20:48:40.137,background,PARALLEL REDO TASK,NULL,NULL,NULL,NULL,0,1,NULL,0,DISPATCHER_QUEUE_SEMAPHORE,46115,DISPATCHER_QUEUE_SEMAPHORE,,0,1,0,NULL,0,0,0,47376,9,0x00000010004428C8,29,30,365,4096,us_english,mdy,7,0,0,0,0,0,0,0,0,2,-1,0,0,0,1,0,0,1,NULL,NULL,NULL,NULL,1,NULL,NULL,0,NULL,0,00000000-0000-0000-0000-000000000000,NULL
10,0,2024-

The above return contains several attributes desribing the query that is being executed. We are not going to get into the specifics of what each of these attributes mean, but please scroll through them to see how much information is available.

The next script will need to be executed in a regular SQL file because it is not supported in Jupyter notebook. Use ctrl+n or Command+n to open a sql file and paste the below script. Don't forget to setup your connection before running your code.

In [5]:
-- Analyze query execution plan for a sample query
SET SHOWPLAN_XML ON;
GO
SELECT * FROM Ms0885011_table WHERE column_1 = 1;
GO
SET SHOWPLAN_XML OFF;
GO

Commands completed successfully.

(1 row affected)

Commands completed successfully.

Total execution time: 00:00:00.380

Microsoft SQL Server 2005 XML Showplan


The above code provides a small diagram that describes how the query works. If you click on different items (like Table Scan) you can see information like CPU cost of the query. We have mostly ran simple queries in this course, but as you begin working and have to write complex and system intensive queries, these tools can provide valuable insight on the performance of those queries.

  

Finally, we are going to enable statistics information for a query. The below cell returns data on the amount of time that it took to perform different parts of the query operation as well as what kind of tasks the query performed. If a query is taking especially long, running it with statistics enabled may provide some insights.

In [6]:
-- View statistics for a sample query
SET STATISTICS IO ON;
SET STATISTICS TIME ON;
GO
SELECT * FROM Ms0885011_table WHERE column_1 = 1;
GO
SET STATISTICS IO OFF;
SET STATISTICS TIME OFF;
GO

Commands completed successfully.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

(0 rows affected)

Table 'Ms0885011_table'. Scan count 1, logical reads 0, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 0 ms.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.005

column_1,column_2


There are no exercises for this module. You will recieve full credit for turning in this notebook after executing all of the above cells correctly and completing the scenario.

## **Scenario**

You are a database administrator for a new animal shelter. The animal shelter needs a database with the following considerations:

**General Info:**

1. The database should be in BCNF, and you **must include a Miro ERD in your repo.**
2. You are provided with two general data categories, animals, and employees. While these will be the main entities in the database, to reach BCNF you will have to normalize the data further than just a handful of tables. **The intended solution has six tables**, not including system tables like Users and Login.
3. Of the six tables you should have, three of them are related to animals, two of them are related to employee data, and the final one is a junction table containing information about both. Consider the junction table "animal data" for the purpose of granting permissions.
4. The database must keep track of the listed data points (below)
5. Each database user should have a login that is connected to the appropriate role or has the correct permissions granted
6. There should be cells in the notebook to facilitate the backup and restoration of the database
7. There are a handful of automation requirements that should be met for the database (See Create 10 for help if you forget your syntax)

**Data Points:**

1. Animal name
2. Animal type (dog, cat, fish, etc...)
3. Animal breed (beagle, tabby, goldfish, etc...)
4. Animal birthdate
5. Animal gender
6. Employee name (first and last)
7. Employee department (volunteer, administration, IT, Vet)
8. Employee hire date
9. Employee salary
10. Employee login (username)
11. For volunteers, store which animal types they work with

**Permissions:**

Each employee is a user for the system. Below is a table of employees and the permissions that they should have. If there is an existing DB role that fits their permission set, use that, otherwise use GRANT to give specific permissions.

| Employee Name | Department | Read Animal Data | Write Animal Data | Read Employee Data | Write Employee Data | Backup Database | Manage Security | Manage DB Structure |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Alice Thompson | Administration | Yes | Yes | Yes | Yes | Yes | Yes | Yes |
| Bob Smith | IT | Yes | Yes | Yes | No | Yes | Yes | Yes |
| Carol Johnson | Vet | Yes | Yes | No | No | No | No | No |
| Daniel Miller | Administration | Yes | Yes | Yes | Yes | No | No | No |
| Emily Clark | Volunteer | Yes | No | No | No | No | No | No |
| Frank Wilson | IT | Yes | No | Yes | No | Yes | No | No |
| Grace Turner | Vet | Yes | Yes | No | No | No | No | No |
| Hannah Young | Administration | Yes | Yes | Yes | Yes | No | No | No |
| Ian Lopez | IT | Yes | No | Yes | No | No | Yes | Yes |
| Julia Evans | Volunteer | Yes | No | No | No | No | No | No<br> |

To properly complete the assignment, you will need to use a combination of role assignments and specific permission granting. To assist with this task, here are the roles that you can add members to (for this assignment):

- db\_owner: Grants full control over the database, including configuration, data, and structure
- db\_datareader: Grants permission to read data from all tables
- db\_datawriter: Grants permission to write (insert, update, delete) data in all tables
- db\_backupoperator: Grants permission to back up the database
- db\_securityadmin: Grants permission to manage security (logins, roles, permissions)
- db\_ddladmin: Grants permissino to execute Data Definition Language (DDL) commands such as creating, altering, or dropping database objects

For example, you should check to see if an employee needs the db\_datareader role so that they can view all tables, or if they just need granted access to specific tables. An example of this would be Emily Clark, she needs to be granted specific access to read tables that contain animal data, but she is not allowed to view employee tables. Emily should be granted specific permissions instead of broad roles.

**Required Data**

_Animals_

| Animal Name | Animal Type | Animal Breed | Birthdate | Gender |
| --- | --- | --- | --- | --- |
| Bella | Dog | Beagle | 2020-05-10 | Female |
| Max | Cat | Tabby | 2019-08-21 | Male |
| Goldie | Fish | Goldfish | 2022-02-14 | Female |
| Rocky | Dog | German Shepherd | 2018-11-01 | Male |
| Whiskers | Cat | Tabby | 2021-03-15 | Female |
| Charlie | Bird | Parrot | 2020-12-05 | Male |
| Luna | Rabbit | Holland Lop | 2021-07-22 | Female |
| Rex | Dog | Beagle | 2017-09-09 | Male |

_Employees_

| Employee Name | Department | Hire Date | Salary | Animal Types Worked With (for Volunteers) |
| --- | --- | --- | --- | --- |
| Alice Thompson | Administration | 2018-02-15 | 75000 | N/A |
| Bob Smith | IT | 2020-06-01 | 68000 | N/A |
| Carol Johnson | Vet | 2019-03-20 | 85000 | N/A |
| Daniel Miller | Administration | 2021-08-10 | 60000 | N/A |
| Emily Clark | Volunteer | 2023-04-12 | N/A | Dogs, Cats |
| Frank Wilson | IT | 2017-11-25 | 72000 | N/A |
| Grace Turner | Vet | 2019-05-30 | 82000 | N/A |
| Hannah Young | Administration | 2022-01-07 | 59000 | N/A |
| Ian Lopez | IT | 2021-09-18 | 69000 | N/A |
| Julia Evans | Volunteer | 2022-06-15 | N/A | Cats, Rabbits |

**Deliverables**

1. Add your completed ERD to your github repo (the folder you are in now) before submitting the assignment
2. Fill all of the below cells, and use the "+ Cell" button to add additional cells as needed. Do not remove cells, only add.
1. For DML and DDL operations you should use full transactions with try/catch implementation
4. **Run all code cells so that I can observe the output.** Because you are running this database locally, I cannot grade if the cells have not been ran.

In [3]:
--Create Database (name the database animal_shelter)
CREATE DATABASE animal_shelter;

: Msg 1801, Level 16, State 3, Line 2
Database 'animal_shelter' already exists. Choose a different database name.

Total execution time: 00:00:00.004

In [4]:
--Create Logins (logins should be firstname_lastname all lower case (alice_thompson))
--You can set a default password for all users if you would like
CREATE LOGIN alice_thompson WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN bob_smith WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN carol_johnson WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN daniel_miller WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN emily_clark WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN frank_wilson WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN grace_turner WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN hannah_young WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN ian_lopez WITH PASSWORD = 'RealPassword1234';
CREATE LOGIN julia_evans WITH PASSWORD = 'RealPassword1234';

Commands completed successfully.

Total execution time: 00:00:00.024

In [5]:
--Create Users (Don't forget to switch to the animal_shelter database)
USE animal_shelter
CREATE USER alice_thompson FOR LOGIN alice_thompson
CREATE USER bob_smith FOR LOGIN bob_smith
CREATE USER carol_johnson FOR LOGIN carol_johnson
CREATE USER emily_clark FOR LOGIN emily_clark
CREATE USER frank_wilson FOR LOGIN frank_wilson
CREATE USER grace_turner FOR LOGIN grace_turner
CREATE USER hannah_young FOR LOGIN hannah_young
CREATE USER ian_lopez FOR LOGIN ian_lopez
CREATE USER julia_evans FOR LOGIN julia_evans


Commands completed successfully.

Total execution time: 00:00:00.013

In [6]:
CREATE TABLE animalTypes
(
    typeID INT IDENTITY(1,1) PRIMARY KEY,
    typeName NVARCHAR(50) NOT NULL
);

CREATE TABLE animalBreeds
(
    breedID INT IDENTITY(1,1) PRIMARY KEY,
    breedName NVARCHAR(50) NOT NULL,
    typeID INT NOT NULL,
    FOREIGN KEY (typeID) REFERENCES AnimalTypes(typeID)
);

CREATE TABLE animals
(
    animalID INT IDENTITY(1,1) PRIMARY KEY,
    name NVARCHAR(50) NOT NULL,
    typeID INT NOT NULL,
    breedID INT NOT NULL,
    birthdate DATE NOT NULL,
    gender NVARCHAR(10) CHECK (gender IN ('Male', 'Female')),
    FOREIGN KEY (typeID) REFERENCES AnimalTypes(typeID),
    FOREIGN KEY (breedID) REFERENCES AnimalBreeds(breedID)
);

CREATE TABLE employees
(
    employeeID INT IDENTITY(1,1) PRIMARY KEY,
    firstName NVARCHAR(50) NOT NULL,
    lastName NVARCHAR(50) NOT NULL,
    department NVARCHAR(20) CHECK (department IN ('Volunteer', 'Administration', 'IT', 'Vet')),
    hireDate DATE NOT NULL,
    salary DECIMAL(10,2) NULL
);

CREATE TABLE volunteerPref
(
    preferenceID INT IDENTITY(1,1) PRIMARY KEY,
    volunteerID INT NOT NULL,
    typeID INT NOT NULL,
    FOREIGN KEY (volunteerID) REFERENCES Employees(employeeID),
    FOREIGN KEY (typeID) REFERENCES AnimalTypes(typeID)
);

CREATE TABLE animalAssignments
(
    assignmentID INT IDENTITY(1,1) PRIMARY KEY,
    employeeID INT NOT NULL,
    animalID INT NOT NULL,
    FOREIGN KEY (employeeID) REFERENCES Employees(employeeID),
    FOREIGN KEY (animalID) REFERENCES Animals(animalID)
);


Commands completed successfully.

Total execution time: 00:00:00.015

In [7]:
--Insert statements for each table
INSERT INTO animalTypes (typeName) VALUES 
('Dog'), ('Cat'), ('Fish'), ('Bird'), ('Rabbit');

INSERT INTO animalBreeds (breedName, typeID) VALUES 
('Beagle', 1), ('Tabby', 2), ('Goldfish', 3), 
('German Shepherd', 1), ('Parrot', 4), ('Holland Lop', 5);

INSERT INTO animals (name, typeID, breedID, birthdate, gender) VALUES
('Bella', 1, 1, '2020-05-10', 'Female'),
('Max', 2, 2, '2019-08-21', 'Male'),
('Goldie', 3, 3, '2022-02-14', 'Female'),
('Rocky', 1, 4, '2018-11-01', 'Male'),
('Charlie', 4, 5, '2020-12-05', 'Male');

INSERT INTO employees (firstName, lastName, department, hireDate, salary) VALUES
('Alice', 'Thompson', 'Administration', '2018-02-15', 75000),
('Bob', 'Smith', 'IT', '2020-06-01', 68000),
('Emily', 'Clark', 'Volunteer', '2023-04-12', NULL),
('Julia', 'Evans', 'Volunteer', '2022-06-15', NULL);

INSERT INTO volunteerPref (volunteerID, typeID) VALUES
(3, 1), (3, 2), (4, 2), (4, 5);

INSERT INTO animalAssignments (employeeID, animalID) VALUES
(1, 1), (2, 2), (3, 3), (4, 4);


(5 rows affected)

(6 rows affected)

(5 rows affected)

(4 rows affected)

(4 rows affected)

(4 rows affected)

Total execution time: 00:00:00.029

In [8]:
--Give users specific roles
EXEC sp_addrolemember 'db_owner', 'alice_thompson';
EXEC sp_addrolemember 'db_backupoperator', 'bob_smith';
EXEC sp_addrolemember 'db_datareader', 'emily_clark';
EXEC sp_addrolemember 'db_datareader', 'julia_evans';



Commands completed successfully.

Total execution time: 00:00:00.010

In [9]:
--Grand users specific permissions
GRANT SELECT ON animals TO emily_clark;
GRANT SELECT ON animals TO julia_evans;
GRANT SELECT, INSERT, UPDATE ON employees TO alice_thompson;
GRANT BACKUP DATABASE TO bob_smith;


Commands completed successfully.

Total execution time: 00:00:00.008

In [ ]:
--Create a back up of the database
BACKUP DATABASE animal_shelter TO DISK = 'C:\Backup\AnimalShelter.bak';



In [ ]:
--Write script to resote the database from the back up created in the previous cell
RESTORE DATABASE animal_shelter FROM DISK = 'C:\Backup\AnimalShelter.bak';


In [10]:
--Create a view that includes the following
--All data from the junction table between animal and employee...and the related employees first and last name
--Name the view appropriately (vs_ prefix)
CREATE VIEW vs_animalAssignments AS
SELECT 
    aea.assignmentID,
    aea.employeeID,
    e.firstName,
    e.lastName,
    aea.animalID
FROM animalAssignments aea
JOIN employees e ON aea.employeeID = e.employeeID;


Commands completed successfully.

Total execution time: 00:00:00.005

In [11]:
--Give everyone that has read access for animal data, but not employee data permission to read the view. (This allows thenm to see which employees work with what animals)
GRANT SELECT ON vs_animalAssignments TO julia_evans;


Commands completed successfully.

Total execution time: 00:00:00.004

In [15]:
--Create a trigger that adds a login and user when a new employee is added to the database
--could not get to function



Commands completed successfully.

Total execution time: 00:00:00.001

| Employee Name      | Department     | Hire Date  | Salary  | Animal Types Worked With (for Volunteers) |
|--------------------|----------------|------------|---------|-------------------------------------------|
| **Mark Taylor**     | Volunteer      | 2023-03-05 | N/A     | Dogs, Birds                               |


In [16]:
--Add the above employee to the database (to fire the trigger)
INSERT INTO employees (firstName, lastName, department, hireDate, salary) VALUES
('Mark', 'Taylor', 'Volunteer', '2023-03-05', NULL);



: Msg 15002, Level 16, State 1, Procedure sp_addlogin, Line 19
The procedure 'sys.sp_addlogin' cannot be executed within a transaction.

: Msg 15002, Level 16, State 1, Procedure sp_grantdbaccess, Line 30
The procedure 'sys.sp_grantdbaccess' cannot be executed within a transaction.

: Msg 15410, Level 11, State 1, Procedure sp_addrolemember, Line 35
User or role 'mark_taylor' does not exist in this database.

: Msg 3609, Level 16, State 1, Line 2
The transaction ended in the trigger. The batch has been aborted.

Total execution time: 00:00:00.003

In [17]:
--View all database users (prefilled to assist in your testing)
EXEC sp_helpuser

Commands completed successfully.

Total execution time: 00:00:00.098

UserName,RoleName,LoginName,DefDBName,DefSchemaName,UserID,SID
alice_thompson,db_owner,alice_thompson,master,dbo,5,0x48B26100C00E8F4F87C36B72AB285A69
bob_smith,db_backupoperator,bob_smith,master,dbo,6,0x4F3E89AA1D16904AA5F9D51352B666B9
carol_johnson,public,carol_johnson,master,dbo,7,0x1B602AA011AF694D8851DC51C34C7498
dbo,db_owner,sa,master,dbo,1,0x01
emily_clark,db_datareader,emily_clark,master,dbo,8,0x3C420D02BAFB52429A9E121A0F2844C2
frank_wilson,public,frank_wilson,master,dbo,9,0xE1D0D692CFB746489C6E74235596E107
grace_turner,public,grace_turner,master,dbo,10,0x70E692089479544CA17067E9035E3E19
guest,public,NULL,NULL,guest,2,0x00
hannah_young,public,hannah_young,master,dbo,11,0x31D043BAD31CBA4EA2EA1F61BDD5D234
ian_lopez,public,ian_lopez,master,dbo,12,0x1578B5886EDC924A86E569F4A634CFD1
